# In this notebook
- Randomized iterative methods (Gower's paper)
- Using them to linear solve matrices AX = B
- ADI 

All are tested very loosely with small matrices. A more formal unit test suite can be found in the project in `../src`.

In [1]:
using Random
using LinearAlgebra
using SparseArrays
using Plots
# using Printf

## Random iterative method (parameterized) 

Solves $Ax = b$

In [2]:
function rand_linsolve(A,b,B, Stype, seed=nothing,verbose=false)
    """
    Solves using randomized kaczmarz.
    Randomly selects a row per iteration to update x with
    
    INPUT:  matrix A mxn, 
            vector b mx1,
            matrix B nxn
            string Stype: "coordvec", ""
            
    OUTPUT: vector x nx1 that solves Ax = b
    """
    m,n = size(A)
    x0 = ones(n)
    sols = [x0] # list of x_0 ... x_n
    errs = zeros(0)
    if seed != nothing
        Random.seed!(seed)
    end
    @assert Stype == "coordvec"
    
    Binv = inv(B)
    xprev = x0
    while(true)
        
        i = rand(1:m)
        
        # S is a unit coordinate vector (temporary)
        S = zeros(m)
        S[i] = 1
        
        C = Binv * A' * S * pinv(S' * A * Binv * A' * S) * S'      
        res = A * xprev - b
        xnew = xprev - C * res
  
        push!(sols, xnew)
        err = norm(A * xnew - b)
        push!(errs,err)
        if err < 1e-5
            break
        end
        
        xprev = xnew
    end
    
    if verbose
        return sols,errs
    else
        return sols[end]
    end
end

rand_linsolve (generic function with 3 methods)

In [3]:
seed = 0

A = rand(Float64,3,5)
b = rand(Float64,3)undefvar printsize not defined
B = I # n x n 
display("text/plain",A)
display("text/plain",b)
display("text/plain",B)

sols,errs = rand_linsolve(A,b, B,"coordvec", seed,true)
println("done")

LoadError: syntax: extra token "printsize" after end of expression

In [4]:
# print(sols[end])

println(size(errs))
# println(errs)
numiters= size(errs)[1]
plot(1:numiters, errs[:,1])

UndefVarError: UndefVarError: errs not defined

In [5]:
function rand_matsolve(A,B,seed=nothing)
    """
    Helper function that stacks together multiple Ax=b solvers
    INPUT: 
            A: matrix mxn
            B: matrix mxk
    OUTPUT: X: matrix such that AX = B
    """
    
    m1,n = size(A)
    m2,k = size(B)
    sol = zeros(n,k)
    @assert m1==m2
    
    s(b) = rand_linsolve(A,b,I,"coordvec", seed)
    return mapslices(s,B,dims=[1])
    
end

rand_matsolve (generic function with 2 methods)

In [6]:

A = rand(Float64,3,5)
B = rand(Float64,3,6)
Bparam = I # n x n
m1,n = size(A)
m2,k = size(B)
sol = zeros(n,k)

X = rand_matsolve(A,B)

display("text/plain", A * X - B)

3×6 Array{Float64,2}:
 -5.97118e-6   -3.45626e-6  -1.89978e-6   …   1.25746e-6   -3.20742e-6 
  1.11022e-16   0.0          1.11022e-16      3.64292e-17   5.55112e-17
  6.81143e-6    7.89557e-6   9.80136e-6      -9.16492e-6    7.78137e-6 

## ADI

Solves $AX - XB = N$ for low rank $N$

In [28]:
function adi(A,B,F,N,p,q)
    m,m2 = size(A)
    n,n2 = size(B)
    
    @assert m==m2
    @assert n==n2
    
    sols = []
    Xprev = zeros((m,n))
    for i = 1:N
        Ahalf = (A - p[i] * I)
        
        
        Bhalf = Xprev * (B - q[i] * I) + F
        
        Xhalf = rand_matsolve(Ahalf,Bhalf)   
        

        Asolve = (B - q[i] * I)
        Bsolve = (A - q[i] * I) * Xhalf - F
        X = (rand_matsolve(Asolve', Bsolve'))'
        Xprev = X 
        push!(sols,X)
    end    
    return sols
end

adi (generic function with 1 method)

In [29]:
A = rand(Float64,5,5)
B = rand(Float64,3,3)
F = rand(Float64,5,3)
N = 5
p = ones(N)
q = ones(N)

sols = adi(A,B,F,N,p,q)
display("text/plain",sols[end])


5×3 Adjoint{Float64,Array{Float64,2}}:
  2.53446e-5  4.40675e-5    3.6082e-6 
  8.23692e-5  0.000146286   7.21577e-5
  2.86763e-5  4.7045e-5     2.82769e-7
 -1.52371e-5  6.44682e-6   -1.31587e-5
  1.63336e-5  4.24438e-5    5.7095e-6 